# Коды обнаружения и исправления ошибок 2

В этой задаче вы продолжите знакомство с некоторыми алгебраическими принципами, лежащими в основе кодов исправления ошибок. В том числе вы познакомитесь с конечными полями, отличными от $\mathbb{Z}_p$, которые повсеместно применяются в кодах и криптографии.

![](jDR8NDD.jpeg)

Рассматриваемые конструкции могут показаться на первый взгляд надуманными или излишне усложнёнными. Но это не так. Вы касаетесь ежедневно всех этих технологий (и их более сложных вариантов) при использовании Flash накопителей, SSD жёстких дисков, шифрования HTTPS и т.п.

*Техническое замечание.* В этой задаче вам может понадобиться проводить вычисления в полях $\mathbb{Z}_p$, где $p$ — простое число. Вычисление обратного элемента для ненулевого $a$ из $\mathbb{Z}_p$ (т.е. такого $b \in \mathbb{Z}_p$, что $a\cdot b \equiv 1 \pmod{p}$) выполняется в Python 3.8 следующим образом: `b = pow(a, -1, p)`; если ваша версия Python более старая, можете воспользоваться подготовленной нами функцией `modinv`: https://pastebin.com/SmmQiLSx. Чтобы избежать неочевидных ошибок переполнения при выполнении вычислений в $\mathbb{Z}_p$, рекомендуем брать остаток от деления после *каждой* операции; например, выражение $\frac{4 + x^3}{a - b + 1}$ в $\mathbb{Z}_7$ вычисляется как `((4 + ((x*x)%7)*x%7) % 7 * pow(((a - b) % 7 + 1) % 7, -1, 7)) % 7`. В частности, для вычисления $x^n$ в $\mathbb{Z}_p$ не используйте код `x**n % p`, который приведёт к ошибкам переполнения при большом $n$. В учебных целях просим вас не пользоваться пакетом `sympy`.

**1. Интерполяция многочленом.** Допишите функцию `polynomial_by_values(xs, ys)`, которая получает на вход в списках `xs` и `ys` одинаковой длины, соответственно, элементы $x_1, x_2, \ldots, x_n$ и $y_1, y_2, \ldots, y_n$ из поля $\mathbb{Z}_{251}$, где все $x_1, x_2, \ldots, x_n$ различны, и возвращает список элементов $c_1, c_2, \ldots, c_n$ из $\mathbb{Z}_{251}$, такой что многочлен $f(x) = \sum_{i=1}^{n} c_i x^{n-i}$ в $\mathbb{Z}_{251}$ удовлетворяет равенствам $f(x_i) = y_i$ для всех $i \in \{1,\ldots,n\}$.


In [7]:
from pandas.core.array_algos.transforms import shift
def mod_inv(a, mod=251):
    if a % mod == 0:
        raise ValueError()

    return pow(a, mod - 2, mod)


def poly_add(p1, p2, mod=251):
    n = max(len(p1), len(p2))
    res = [0] * n

    for i in range(n):
        c1 = p1[i] if i < len(p1) else 0
        c2 = p2[i] if i < len(p2) else 0
        res[i] = (c1 + c2) % mod

    return res


def poly_mul_monomial(p, coef, power, mod=251):
    res = [0] * (len(p) + power)
    for i in range(len(p)):
        res[i + power] = (p[i] * coef) % mod

    return res


def poly_mul(p1, p2, mod=251):
    res = [0] * (len(p1) + len(p2) - 1)
    for i in range(len(p1)):
        for j in range(len(p2)):
            res[i + j] = (res[i + j] + p1[i] * p2[j]) % mod
    return res


def polynomial_by_values(xs, ys):
    mod = 251
    n = len(xs)
    if n == 0 or len(ys) != n:
        raise ValueError()

    p = [0] * n
    for j in range(n):
        Lj = [1]
        for m in range(n):
            if m == j:
                continue
            denom = (xs[j] - xs[m]) % mod
            inv_denom = mod_inv(denom, mod)
            factor = [(-xs[m]) % mod, 1]
            factor = [(coef * inv_denom) % mod for coef in factor]
            Lj = poly_mul(Lj, factor, mod)

        Lj = [(coef * ys[j]) % mod for coef in Lj]
        p = poly_add(p, Lj, mod)

    p = p[:n]
    p = list(reversed(p))

    return p


**2. Коррекция ошибок.** 
Пусть кодируемые данные представлены последовательностью элементов из $\mathbb{Z}_{251}$ и каждый блок данных из 40 элементов кодируется отдельно (на практике вместо $\mathbb{Z}_{251}$ обычно используется поле размера $256$, чтобы каждый элемент занимал один байт; смотрите следующие подзадачи). Для блока $b_1, b_2, \ldots, b_{40}$, где $b_i \in \mathbb{Z}_{251}$ для $i \in \{1,\ldots,40\}$, пусть $f(x) = \sum_{i=1}^{40} d_i x^{40-i}$ — это многочлен из $\mathbb{Z}_{251}[x]$, такой что $b_i = f(i-1)$ в $\mathbb{Z}_{251}$ для каждого $i \in \{1,\ldots,40\}$. Блок $b_1, b_2, \ldots, b_{40}$ кодируется последовательностью $c_1, c_2, \ldots, c_{44}$, такой что $c_i = f(i-1)$ в $\mathbb{Z}_{251}$ для каждого $i \in \{1,\ldots,44\}$; таким образом, кодовая последовательность $c_1, c_2, \ldots, c_{44}$ получена приписыванием справа к блоку $b_1, b_2, \ldots, b_{40}$ элементов $f(40)$, $f(41)$, $f(42)$ и $f(43)$.

Допишите функцию, которая по последовательности $c_1, c_2, \ldots, c_{44}$ возвращает исходную последовательность $b_1, b_2, \ldots, b_{40}$, даже если кто-то подменил один *или два* элемента $c_i$ на другое значение из $\mathbb{Z}_{251}$ (если изменено больше двух $c_i$, вашему коду позволено этого не замечать и вести себя как угодно). Решение должно отрабатывать все тесты не дольше, чем за минуту.

In [8]:
mod = 251
n = 44

def inverse(a):
    return pow(a, mod - 2, mod)


def solve_linear(a, b):
    n, m = len(a), len(a[0])
    aug = [row[:] + [b[i]] for i, row in enumerate(a)]
    r = 0

    for c in range(m):
        p = next((i for i in range(r, n) if aug[i][c] % mod), None)
        if p is None: continue

        aug[r], aug[p] = aug[p], aug[r]
        inv_p = inverse(aug[r][c])

        for j in range(c, m + 1):
            aug[r][j] = (aug[r][j] * inv_p) % mod
        for i in range(n):
            if i == r: continue

            f = aug[i][c]
            if f == 0: continue
            for j in range(c, m + 1):
                aug[i][j] = (aug[i][j] - f * aug[r][j]) % mod

        r += 1

    x = [0] * m
    for i in range(n):
        lead = next((j for j, v in enumerate(aug[i][:m]) if v), None)
        if lead is not None:
            x[lead] = aug[i][m]

    return x


def poly_division(num, den):
    num = num[:]
    while num and num[-1] == 0:
        num.pop()
    while den and den[-1] == 0:
        den.pop()

    dnum, dden = len(num) - 1, len(den) - 1
    if dden < 0:
        raise ValueError()

    q = [0] * (max(dnum - dden + 1, 0))
    inv_lc = inverse(den[-1])

    while dnum >= dden:
        c = num[-1] * inv_lc % mod
        s = dnum - dden
        q[s] = c

        for k in range(dden + 1):
            num[s + k] = (num[s + k] - c * den[k]) % mod
        while num and num[-1] == 0:
            num.pop()
        dnum = len(num) - 1

    return q, num


def correct_errors(code):
    if len(code) != n:
        raise ValueError()

    xs = list(range(n))
    a, b = [], []

    for x, c in zip(xs, code):
        row = [pow(x, d, mod) for d in range(42)]
        row += [(-c * x) % mod, (-c * x * x) % mod]
        a.append(row)
        b.append(c % mod)

    solution = solve_linear(a, b)

    q = solution[:42]
    e1, e2 = solution[42], solution[43]
    f, rem = poly_division(q, [1, e1, e2])
    res = []

    for x in range(40):
        val = 0
        p   = 1
        for c in f:
            val = (val + c * p) % mod
            p   = (p * x) % mod
        res.append(val)

    return res


**3. Поле размера $\mathbf{2^{32}}$.** В этой и следующей подзадачах на примере поля размера $2^{32}$ будет рассмотрена схема построения полей размера $2^n$, обозначаемых $\mathrm{GF}(2^n)$ или $\mathbb{F}_{2^n}$ (построение других полей $\mathbb{F}_{2^n}$, в том числе $\mathbb{F}_{256}$, аналогично построению $\mathbb{F}_{2^{32}}$).

Поле $\mathbb{F}_{2^{32}}$ похоже на $\mathbb{Z}_p$ (где $p$ — простое), но строится на многочленах, а не на целых числах. Для построения $\mathbb{F}_{2^{32}}$ зафиксируем в $\mathbb{Z}_2[x]$ многочлен $g(x) = 1 + x + x^2 + x^{22} + x^{32}$ (можно брать и другие многочлены $g(x)$ степени 32, но получающаяся конструкция тогда не обязательно будет полем; см. далее); роль этого многочлена аналогична роли простого числа $p$ при построении поля $\mathbb{Z}_p$. Элементы $\mathbb{F}_{2^{32}}$ — это многочлены $\sum_{i=1}^{32} b_i x^{i-1}$ из $\mathbb{Z}_2[x]$ степени меньше 32; каждый такой многочлен может быть записан последовательностью из 32 своих коэффициентов $b_1, b_2, \ldots, b_{32} \in \mathbb{Z}_2$, как раз умещающихся в четыре байта. Очевидно, $\mathbb{F}_{2^{32}}$ имеет размер $2^{32}$. Сложение в $\mathbb{F}_{2^{32}}$ — это обычное сложение многочленов в $\mathbb{Z}_2[x]$. Произведение двух элементов $a$ и $b$ из $\mathbb{F}_{2^{32}}$, обозначаемое $a * b$, — это остаток от деления многочлена $a(x)\cdot b(x)$ на $g(x)$. Например, $(x ^2+ x^{12}) * x^{20} = 1+x+x^2$.

Покажите, что $\mathbb{F}_{2^{32}}$ является коммутативным и ассоциативным кольцом с единицей. Такое кольцо является полем, если каждый ненулевой элемент $a$ имеет (обратный) элемент $b$, такой что $a * b = 1$. Для $g(x) = 1 + x + x^2 + x^{22} + x^{32}$ известно, что это так. Допишите функцию, которая по ненулевому многочлену $a(x) = \sum_{i=1}^{32} a_i x^{i-1}$ из $\mathbb{Z}_2[x]$, заданному списком битов $a_1, a_2, \ldots, a_{32}$, возвращает список битов $b_1, b_2, \ldots, b_{32}$, такой что многочлен $b(x) = \sum_{i=1}^{32} b_i x^{i-1}$ является обратным к $a(x)$ в $\mathbb{F}_{2^{32}}$.

*Подсказка.* Если для каких-то многочленов $u(x), v(x) \in \mathbb{Z}_2[x]$ выполняется $u(x)a(x) + v(x)g(x) = 1$, то $u(x)a(x) = -v(x)g(x) + 1$ и поэтому остаток от деления многочлена $u(x)a(x)$ на $g(x)$ равен $1$. Всегда ли существуют такие $u(x)$ и $v(x)$? Воспользуйтесь алгоритмом Евклида (в качестве подсказки вам дана реализация алгоритма Евклида для целых чисел: https://pastebin.com/7EdVrzuU).

In [9]:
irreducible = (1 << 32) | (1 << 22) | (1 << 2) | (1 << 1) | 1
deg = 32

def degree(p: int) -> int:
    return p.bit_length() - 1


def polymod(p: int) -> int:
    while p.bit_length() > 32:
        shift = p.bit_length() - 33
        p ^= irreducible << shift

    return p


def polymul(a: int, b: int) -> int:
    res = 0
    while b:
        if b & 1:
            res ^= a
        b >>= 1
        a <<= 1

    return res


def poly_gcdex(a: int, b: int):
    x0, x1 = 1, 0
    y0, y1 = 0, 1

    while b:
        q = 0
        da, db = degree(a), degree(b)

        while da >= db:
            shift = da - db
            q ^= 1 << shift
            a ^= b << shift
            da = degree(a)

        x0, x1 = x1, x0 ^ polymul(q, x1)
        y0, y1 = y1, y0 ^ polymul(q, y1)
        a, b  = b, a

    return a, x0, y0


def bits_to_int(bits):
    p = 0
    for i, bit in enumerate(bits):
        if bit:
            p |= 1 << i
    return p


def int_to_bits(x):
    return [(x >> i) & 1 for i in range(32)]


def field_inverse(a_bits):
    if len(a_bits) != 32 or all(b == 0 for b in a_bits):
        raise ValueError()

    a = bits_to_int(a_bits)
    g = (1 << deg) | irreducible

    gcd, u, _ = poly_gcdex(a, g)
    if gcd != 1:
        raise ValueError()

    inv = polymod(u)
    return int_to_bits(inv)


**4. Поля и кольца размера $\mathbf{2^{32}}$.** Описанную в предыдущей подзадаче конструкцию можно провести для любого многочлена $g(x)$ степени 32. Но всегда ли полученное кольцо будет полем? Допишите функцию, которая принимает на вход многочлен $g(x) = \sum_{i=1}^{33} c_i x^{i-1}$ из $\mathbb{Z}_2[x]$ степени 32, заданный списком битов $c_1, c_2, \ldots, c_{33}$, и возвращает `True`, если построенное с помощью этого многочлена кольцо из предыдущей подзадачи является полем, и `False` в противном случае. Обоснуйте, почему ваш код корректен. Код должен проходить все тесты не дольше, чем за минуту.

In [10]:
def generates_field(c):
    if len(c) != 33 or c[-1] != 1:
        raise ValueError

    g = bits_to_int(c)


    def mod_local(p: int) -> int:
        while p.bit_length() - 1 >= deg:
            shift = p.bit_length() - 1 - deg
            p ^= g << shift

        return p

    def frob_pow(power):
        y = 2
        for _ in range(power):
            y = mod_local(polymul(y, y))

        return y

    if frob_pow(32) != 2:
        return False

    for d in (1, 2, 4, 8, 16):
        test = frob_pow(d) ^ 2
        if poly_gcdex(g, test)[0] != 1:
            return False

    return True

## Тестирование

Осталось проверить, проходят ли ваши решения первоначальное тестирование. Выполните весь код (Cell -> Run All или Runtime -> Run All или другим способом, работающим в вашей среде) и посмотрите вердикт внизу страницы. В тестирующем коде разбираться не нужно!

Для демонстрации применения кодов мы подготовили самодельный QR-код. Наша QR-картинка кодирует 44 числа, каждое от 0 до 250. Эти 44 числа кодируют сообщение длины 40 с помощью кода коррекции ошибок, как в подзадаче 2. Каждое из этих 44-х чисел записано в одной из 44-х ячеек на картинке; 4 левые верхние ячейки и правая нижняя ячейка не используются и могут содержать что угодно (для красоты или других функций). Каждое число 0,1,...,250 кодируется с помощью 8 битов. Эти биты записываются чёрными и белыми цветами как на схеме ниже: 0 — чёрный, 1 — белый (наш код несколько расточительный, потому что кодирует 8 битов с помощью 9 клеток, левая верхняя клетка всегда чёрная).

![](TOP3boj.png)

В качестве примера в тестах с помощью вашего решения подзадачи 2 декодируется следующая повреждённая картинка (область повреждения в правом нижнем углу). Результат расшифровки выводится внизу после всех вердиктов по тестам.

![](nnMnYCy.png)

In [11]:
import numpy as np, itertools
import imageio.v2 as imageio

def decode_int251(img, i, j):
    res = 0
    for x, y in itertools.product(range(3), range(3)):
        res *= 2
        avg = 0   # среднее значение 3х3 пикселя img[9i+3x..9i+3x+2, 9j+3y..9j+3y+2]
        for u, v in itertools.product(range(3), range(3)):
          rgb = img[9 * i + 3 * x + u, 9 * j + 3 * y + v]
          avg += 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]

        if avg / 9 > 127:  # пиксель белый, значит бит = 1
            res += 1
    return min(250, res)

def pseudoQR_image_to_block44(url):
    img = imageio.imread(url)
    encoded = []
    for i, j in itertools.product(range(7), range(7)):
        if (i > 1 or j > 1) and (i != 6 or j != 6):  # не верхний 2x2 и не нижний 1x1 угол
            encoded.append(decode_int251(img, i, j))
    return encoded

def decode_pseudoQR():
    encoded = pseudoQR_image_to_block44('3L4jxxj.png')
    decoded = correct_errors(encoded)
    if sum(decoded) == 3389:  # простая проверка, что ваш код всё декодировал верно
        print('Декодированная картинка: ' + "".join([chr(c) for c in decoded]))
    else:
        print('Не удалось корректно декодировать картинку')

def test_all():
    xs = list(range(50)) + list(range(60, 110)) + list(range(120, 220))
    ys = [23, 140, 45, 121, 235, 87, 171, 232, 243, 12, 51, 110, 12, 214, 
          173, 173, 56, 17, 175, 215, 91, 63, 114, 248, 94, 185, 216, 193, 
          25, 247, 98, 158, 166, 157, 231, 139, 89, 188, 127, 162, 162, 81, 
          188, 121, 42, 128, 94, 175, 206, 124, 54, 52, 217, 55, 127, 121, 
          160, 228, 140, 66, 209, 82, 206, 102, 152, 211, 151, 134, 122, 
          150, 83, 145, 223, 46, 148, 247, 233, 181, 125, 242, 180, 74, 15, 
          72, 140, 81, 179, 75, 82, 40, 113, 63, 96, 218, 93, 209, 49, 63, 
          193, 9, 214, 62, 127, 17, 207, 12, 120, 186, 187, 227, 173, 189, 
          162, 25, 112, 0, 120, 236, 65, 184, 193, 188, 114, 107, 54, 11, 
          144, 144, 88, 181, 113, 53, 3, 137, 61, 100, 213, 156, 38, 174, 
          9, 125, 41, 179, 154, 154, 202, 10, 151, 176, 239, 235, 65, 2, 103, 
          125, 106, 199, 246, 74, 51, 143, 54, 65, 204, 20, 168, 225, 213, 
          92, 250, 223, 79, 62, 135, 233, 50, 181, 52, 37, 22, 230, 182, 62, 
          30, 20, 17, 142, 86, 121, 128, 150, 124, 214, 209, 223, 144, 91, 144, 235]
    
    p = polynomial_by_values(xs, ys)
    got = []
    for x in xs:
        acc = itertools.accumulate(p, lambda a, c: ((a * x)%251 + c)%251) #схема Горнера
        got.append(list(acc)[-1])
    if len(p) > len(xs) or got != ys:
        print('Первое задание не прошло тест')
    else:
        print('Первое задание прошло все тесты')
    
    block = ys[0:40]
    cblock = block + [146, 134, 45, 80]   
    succ = True
    for i, j in [(11,11), (2,39), (13,42)]:
        cblock[i] = (cblock[i] + 153 + i) % 251
        cblock[j] = (cblock[j] + 33 + j) % 251
        got = correct_errors(cblock)
        if got != block:
            succ = False
        cblock[i] = (cblock[i] - 153 - i) % 251
        cblock[j] = (cblock[j] - 33 - j) % 251
    if not succ:
        print('Второе задание не прошло тест')
    else:
        print('Второе задание прошло все тесты')

    tests = [ ([0], [0]), ([2, 12], [0, 1, 20, 21, 22, 23, 25, 26, 28, 29, 31]), 
             ([0, 1, 20, 21, 22, 23, 25, 26, 28, 29, 31], [2, 12]), 
             ([0, 1, 5, 6, 7, 14, 17, 25, 30], 
              [0, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 23, 24, 26, 28, 30, 31]),
            ([0, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 18, 19, 20, 21, 23, 24, 26, 28, 30, 31],
              [0, 1, 5, 6, 7, 14, 17, 25, 30])] 
    
    succ = True
    for t in tests:
        coef1 = [1 if x in t[0] else 0 for x in range(32)]
        coef2 = [1 if x in t[1] else 0 for x in range(32)]
        res = field_inverse(list(coef1))
        if list(res) != list(coef2):
            print('Третье задание не прошло тест: обратный для', list(coef1), '=', list(coef2))
            succ = False
            break
    if succ:
        print('Третье задание прошло все тесты')

    good_polies = [(32,22,2,1,0), 
                   (32,22,21,20,18,17,15,13,12,10,8,6,4,1,0), 
                   (32,27,26,25,24,23,22,17,13,11,10,9,8,7,2,1,0)]
    bad_polies = [(32,), (32,20,3), (32,3,2,1,0), (32,23,17,16,14,10,8,7,6,5,3,0), (32,0)]
    obstacle = []
    for p in good_polies:
        coef = [1 if x in p else 0 for x in range(33)]
        if obstacle == [] and not generates_field(list(coef)):
            obstacle = list(coef)
    for p in bad_polies:
        coef = [1 if x in p else 0 for x in range(33)]
        if obstacle == [] and generates_field(list(coef)):
            obstacle = list(coef)
    if obstacle != []:
        print('Четвёртое задание не прошло тест:', obstacle)
    else:
        print('Четвёртое задание прошло все тесты')
    decode_pseudoQR()

In [12]:
test_all()

Первое задание прошло все тесты
Второе задание прошло все тесты
Третье задание прошло все тесты
Четвёртое задание прошло все тесты
Декодированная картинка: Have fun & learn algebra  bit.ly/3rw788f
